# Preparation  
## Configuring AWS environment  
https://hands-on.cloud/introduction-to-boto3-library/#configuring-aws-environment  

In [1]:
import os
import networkx as nx
import boto3
import paramiko

# Run arguments
host_os = 'linux_mac' # or 'windows'
data_dir = '/home/rony/services/milestones_chains/data/'
data_file_name = 'EMS_DCMA_DD_23.08.graphml'
data_path = os.path.join(data_dir, data_file_name)
experiment = 'EMS_DCMA_DD_23_08'
tasks = 'tdas'
results = 'prt'

## AWS
# IPs
resultsIP = '172.31.10.240'
serviceIP = '172.31.15.123'
# Compute instance
key_file_name = 'ds_eu_west2_2.pem'
if host_os=='windows':
    key_file_name = 'C:\\Users\\username\\.aws\\ds_eu_west2_2.pem'
else:
    key_file_name = '~/.aws/ds_eu_west2_2.pem'
INSTANCE_ID = 'i-0586e11281d4b02a2'
AWS_REGION = "eu-west-2"
EC2_RESOURCE = boto3.resource('ec2', region_name=AWS_REGION)
EC2_CLIENT = boto3.client('ec2', region_name=AWS_REGION)

# Storage bucket
profile_name = 'ds_sandbox'
data_bucket = 'programsdatabucket'
results_bucket = 'chainsresults'
session = boto3.session.Session(profile_name=profile_name)
S3_CLIENT = session.client('s3')
S3_RESOURCE = boto3.resource('s3')

In [2]:
# Upload data to an S3 bucket
S3_CLIENT.upload_file(data_path, data_bucket, data_file_name)

In [4]:
# Start compute instance
instance = EC2_RESOURCE.Instance(INSTANCE_ID)
instance.start()
print(f'Starting EC2 instance: {instance.id}')
instance.wait_until_running()
print(f'EC2 instance "{instance.id}" has been started')

instance: ec2.Instance(id='i-0586e11281d4b02a2')
Starting EC2 instance: i-0586e11281d4b02a2
EC2 instance "i-0586e11281d4b02a2" has been started


In [9]:
# Connect to instance to run process 
ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
privkey = paramiko.RSAKey.from_private_key_file('ds_eu_west2_2.pem') # Works only with local pem file
ssh.connect(hostname=serviceIP, username='ubuntu', pkey=privkey)

In [13]:
#process_statement = "python3 remote_run_test.py"
ssh.exec_command('cd milestones_chains')
process_statement = 'python3 ./services/milestones_chains/run_pipelines.py'
process_statement = 'cd services/milestones_chains && python3 service.py {f} {e} {t} {r}'\
.format(f=data_file_name, e=experiment, t=tasks, r=results)
stdin, stdout, stderr = ssh.exec_command(process_statement)
if stderr:
    print('Run attempt encountered error:\n', stderr.readlines())
elif stdout:
    print(stdout.readlines())
    # stdout = stdout.readlines()[0]
    # stdout = stdout.replace('\n', '')
    if stdout == 'completed':
        print(stdout)
        # Stop instance 
        response = EC2_CLIENT.stop_instances(InstanceIds=[INSTANCE_ID], DryRun=False)
        # Delete data from S3 bucket
        s3_resource.Object(data_bucket, data_file_name).delete()

Run attempt encountered error:
 ['Traceback (most recent call last):\n', '  File "/home/ubuntu/.local/lib/python3.8/site-packages/mysql/connector/connection_cext.py", line 246, in _open_connection\n', '    self._cmysql.connect(**cnx_kwargs)\n', '_mysql_connector.MySQLInterfaceError: Too many connections\n', '\n', 'During handling of the above exception, another exception occurred:\n', '\n', 'Traceback (most recent call last):\n', '  File "build_chains.py", line 8, in <module>\n', '    from db_tables import *\n', '  File "/home/ubuntu/services/milestones_chains/modules/db_tables.py", line 8, in <module>\n', '    from config import *\n', '  File "/home/ubuntu/services/milestones_chains/modules/config.py", line 56, in <module>\n', '    conn = mysql.connect(**conn_params)\n', '  File "/home/ubuntu/.local/lib/python3.8/site-packages/mysql/connector/__init__.py", line 272, in connect\n', '    return CMySQLConnection(*args, **kwargs)\n', '  File "/home/ubuntu/.local/lib/python3.8/site-package

In [ ]:
response = EC2_CLIENT.stop_instances(InstanceIds=[INSTANCE_ID], DryRun=False)